# **IMPORT LIBS**

In [102]:
import numpy as np
from openrec.tf1.legacy import ImplicitModelTrainer
from openrec.tf1.legacy.utils.evaluators import ImplicitEvalManager
from openrec.tf1.legacy.utils import ImplicitDataset
from openrec.tf1.legacy.recommenders import CML, BPR, PMF
from openrec.tf1.legacy.utils.evaluators import AUC
from openrec.tf1.legacy.utils.samplers import PairwiseSampler
import numpy as np
import pandas as pd
import scipy.sparse as sps
import os
import pickle
import random


# **GENERATE THE DATASET**

In [103]:
seed = 2384795
np.random.seed(seed=seed)

folder_name = f"../Dataset/"

if os.path.exists(folder_name) == False:
    os.makedirs(folder_name)

## Load the dataset

In [104]:
file_path = 'yahoo_ymusic_v1/ydata-ymusic-rating-study-v1_0-train.txt'

# Load the training set into a DataFrame
df_train = pd.read_csv(folder_name+file_path, sep='\t',names=["UserID","SongID","Rating"], header=None)  # sep='\t' for tab-separated values
df_train.head(10)

,UserID,SongID,Rating
0,1,14,5
1,1,35,1
2,1,46,1
3,1,83,1
4,1,93,1
5,1,94,1
6,1,153,5
7,1,170,4
8,1,184,5
9,1,194,5


## Convert to implicit

"We treat items rated greater than or equal to 4 as relevant, and others as irrelevant, as suggested by prior literature."

In [105]:
POSITIVE_THRESHOLD = 4
df_train['ImplicitRating'] = np.where(df_train['Rating'] >= POSITIVE_THRESHOLD, 1, 0)

df_train.head(10)

,UserID,SongID,Rating,ImplicitRating
0,1,14,5,1
1,1,35,1,0
2,1,46,1,0
3,1,83,1,0
4,1,93,1,0
5,1,94,1,0
6,1,153,5,1
7,1,170,4,1
8,1,184,5,1
9,1,194,5,1


## Check the number of users and items in the training set

"The training set contains 300K ratings given by 15.4K users against 1K songs through natural interactions."

In [106]:
min_user = df_train["UserID"].min()
max_user = df_train["UserID"].max()

min_item = df_train["SongID"].min()
max_item = df_train["SongID"].max()

max_item, max_user

(1000, 15400)

# **GET UNBIASED TESTSET**

## Load the unbiased testset and convert it to implicit

In [107]:
file_path = '../Dataset/yahoo_ymusic_v1/ydata-ymusic-rating-study-v1_0-test.txt'
df_test = pd.read_csv(file_path, sep='\t',names=["UserID","SongID","Rating"], header=None)  # sep='\t' for tab-separated values
df_test['ImplicitRating'] = np.where(df_test['Rating'] >= POSITIVE_THRESHOLD, 1, 0)
df_test.head(10)

,UserID,SongID,Rating,ImplicitRating
0,1,49,1,0
1,1,126,1,0
2,1,138,1,0
3,1,141,1,0
4,1,177,1,0
5,1,268,3,0
6,1,511,1,0
7,1,587,1,0
8,1,772,5,1
9,1,941,1,0


## Check the number of users and items in the training set

"The testing set is collected by asking a subset of 5.4K users to rate 10 randomly selected songs."

In [108]:
df_test["UserID"].max(), df_test["SongID"].max(), int(df_test.shape[0]/df_test["UserID"].max())

(5400, 1000, 10)

## Filter unbiased testset

"We filter the testing set by retaining users who have at least a relevant and an irrelevant song in the testing set and two relevant songs in the training set."

In [109]:
# Select users with at least an irrelevant song in the unbiased testset
usersWithNegativeInteractionInTest = df_test[df_test["ImplicitRating"] == 0]["UserID"].unique()

# Select UserID of users with at least a relevant song in testset
usersWithPositiveInteractionInTest = df_test[df_test["ImplicitRating"] == 1]["UserID"].unique()

# Select UserID of users with at least two relevant song in trainset
usersWithTwoPositiveInteractions = df_train[df_train["ImplicitRating"] == 1].groupby("UserID").filter(lambda x: len(x) >= 2)['UserID'].unique()

# Compute the intersection
set1 = set(usersWithNegativeInteractionInTest)
set2 = set(usersWithPositiveInteractionInTest)
set3 = set(usersWithTwoPositiveInteractions)
valid_users_testset = set1 & set2 & set3

# Filter the testset
df_test_filtered = df_test[df_test["UserID"].isin(valid_users_testset)]

"2296 users satisfy these requirements."

In [110]:
len(valid_users_testset)

2296

## Shape the unbiased test set

In [111]:
# From the dataframe, for each row where ImplicitRating is 1, append [userID, itemID] to unbiased_pos_test_set
# and for each row where ImplicitRating is 0, append [userID, itemID] to unbiased_neg_test_set

unbiased_pos_test_set = df_test_filtered[df_test_filtered["ImplicitRating"] == 1][["UserID", "SongID"]].values
unbiased_neg_test_set = df_test_filtered[df_test_filtered["ImplicitRating"] == 0][["UserID", "SongID"]].values

## Save unbiased test set

In [112]:
unbiased_pos_test_set_df = pd.DataFrame(unbiased_pos_test_set)
unbiased_neg_test_set_df = pd.DataFrame(unbiased_neg_test_set)

unbiased_pos_test_set_df.columns = ["user_id","item_id"]
unbiased_neg_test_set_df.columns = ["user_id","item_id"]

structured_data_pos_test_set_unbiased = unbiased_pos_test_set_df.to_records(index=False)
structured_data_neg_test_set_unbiased = unbiased_neg_test_set_df.to_records(index=False)

np.save(folder_name + "unbiased-test_arr_pos.npy", structured_data_pos_test_set_unbiased)
np.save(folder_name + "unbiased-test_arr_neg.npy", structured_data_neg_test_set_unbiased)

# **GET BIASED TESTSET**

In [113]:
file_path = 'yahoo_ymusic_v1/ydata-ymusic-rating-study-v1_0-train.txt'

# Load the training set into a DataFrame
df_train = pd.read_csv(folder_name+file_path, sep='\t',names=["UserID","SongID","Rating"], header=None)  # sep='\t' for tab-separated values
df_train.head(10)

,UserID,SongID,Rating
0,1,14,5
1,1,35,1
2,1,46,1
3,1,83,1
4,1,93,1
5,1,94,1
6,1,153,5
7,1,170,4
8,1,184,5
9,1,194,5


In [114]:
df_train['ImplicitRating'] = np.where(df_train['Rating'] >= POSITIVE_THRESHOLD, 1, 0)


In [115]:
df_train = df_train[df_train["UserID"].isin(valid_users_testset)]

In [116]:
df_train.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 58799 entries, 0 to 129178
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   UserID          58799 non-null  int64
 1   SongID          58799 non-null  int64
 2   Rating          58799 non-null  int64
 3   ImplicitRating  58799 non-null  int64
dtypes: int64(4)
memory usage: 2.2 MB


## Extract the biased test set and shape it

"We additionally held out a biased testing set (biased-testing) from the training set by randomly sampling 300 songs for each user."

In [117]:
# Precompute, for each user, the list of songs with a relevant rating
user_positive_ratings = df_train[df_train["ImplicitRating"] == 1].groupby("UserID")["SongID"].apply(set)

# Initialize the range of indexes for the items
items_ids = np.arange(min_item, max_item + 1)
# Set the number of songs for each user
SONGS_FOR_BIASED_TEST = 300

#IPOTESI MAN

pos_test_set = []
neg_test_set = []

for user_id in valid_users_testset:
    np.random.shuffle(items_ids)
    test_items = set(items_ids[-SONGS_FOR_BIASED_TEST:])
    pos_ids = user_positive_ratings.get(user_id, set()) & test_items

    #set those to 0 so that they will no longer be used in training set
    df_train.loc[(df_train['SongID'].isin(pos_ids)) & (df_train['UserID'] == user_id), 'ImplicitRating'] = 0

    for id in test_items:
        if id in pos_ids:
            pos_test_set.append([user_id, id])
        else:
            neg_test_set.append([user_id, id])

pos_test_set = np.array(pos_test_set)
neg_test_set = np.array(neg_test_set)

KeyboardInterrupt: 

## Save the biased test set

In [ ]:
pos_test_set_df = pd.DataFrame(pos_test_set)
neg_test_set_df = pd.DataFrame(neg_test_set)

pos_test_set_df.columns = ["user_id","item_id"]
neg_test_set_df.columns = ["user_id","item_id"]

structured_data_pos_test_set = pos_test_set_df.to_records(index=False)
structured_data_neg_test_set = neg_test_set_df.to_records(index=False)

np.save(folder_name + "biased-test_arr_pos.npy", structured_data_pos_test_set)
np.save(folder_name + "biased-test_arr_neg.npy", structured_data_neg_test_set)

## **STORE TRAINSET**

## Take couples user-item filtering out the irrelevant ones

In [ ]:
# Only take the couples (user, item) with relevant rating
new_df = df_train[df_train['ImplicitRating'] != 0]
new_df = new_df.drop(columns=['Rating', 'ImplicitRating'])

# Define a dictionary for renaming columns
rename_dict = {
    'UserID': 'user_id',
    'SongID': 'item_id'
}

# Rename the columns
new_df = new_df.rename(columns=rename_dict)

# Convert the DataFrame to a structured array
structured_data = new_df.to_records(index=False) 

In [ ]:
train_data = structured_data

## Save the training set

In [ ]:
np.save(folder_name + "training_arr.npy", train_data)

# **MODEL CHOICE**

In [ ]:
raw_data = dict()
raw_data['train_data'] = np.load(folder_name + "training_arr.npy")
raw_data['max_user'] = 15401
raw_data['max_item'] = 1001
batch_size = 8000
test_batch_size = 1000
display_itr = 1000

train_dataset = ImplicitDataset(raw_data['train_data'], raw_data['max_user'], raw_data['max_item'], name='Train')

MODEL_CLASS = CML
MODEL_PREFIX = "cml"
DATASET_NAME = "yahoo"
OUTPUT_FOLDER = "./Output/"
OUTPUT_PATH = OUTPUT_FOLDER + MODEL_PREFIX + "-" + DATASET_NAME + "/"
OUTPUT_PREFIX = str(OUTPUT_PATH) + str(MODEL_PREFIX) + "-" + str(DATASET_NAME)


if os.path.exists(OUTPUT_PATH) == False:
    os.makedirs(OUTPUT_PATH)


# **TRAIN THE MODEL**

In [ ]:
# Avoid tensorflow using cached embeddings
import tensorflow as tf
tf.compat.v1.reset_default_graph()


model = MODEL_CLASS(batch_size=batch_size, max_user=train_dataset.max_user(), max_item=train_dataset.max_item(), 
    dim_embed=50, l2_reg=0.001, opt='Adam', sess_config=None)
sampler = PairwiseSampler(batch_size=batch_size, dataset=train_dataset, num_process=4)
model_trainer = ImplicitModelTrainer(batch_size=batch_size, test_batch_size=test_batch_size,
                                     train_dataset=train_dataset, model=model, sampler=sampler,
                                     eval_save_prefix=OUTPUT_PATH + DATASET_NAME,
                                     item_serving_size=500)
auc_evaluator = AUC()


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






2024-04-05 16:12:29.587918: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2024-04-05 16:12:29.594707: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 4192029999 Hz
2024-04-05 16:12:29.595095: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55feb119e2a0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-04-05 16:12:29.595115: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


In [ ]:
model_trainer.train(num_itr=10001, display_itr=display_itr)

== Start training with FULL evaluation ==
[Itr 100] Finished
[Itr 200] Finished
[Itr 300] Finished
[Itr 400] Finished
[Itr 500] Finished
[Itr 600] Finished
[Itr 700] Finished
[Itr 800] Finished
[Itr 900] Finished
[Itr 1000] Finished
INFO:tensorflow:./Output/cml-yahoo/yahoo-1000 is not in all_model_checkpoint_paths. Manually adding it.
[Itr 1000] loss: 1758.855740
[Itr 1100] Finished
[Itr 1200] Finished
[Itr 1300] Finished
[Itr 1400] Finished
[Itr 1500] Finished
[Itr 1600] Finished
[Itr 1700] Finished
[Itr 1800] Finished
[Itr 1900] Finished
[Itr 2000] Finished
INFO:tensorflow:./Output/cml-yahoo/yahoo-2000 is not in all_model_checkpoint_paths. Manually adding it.
[Itr 2000] loss: 649.815192
[Itr 2100] Finished
[Itr 2200] Finished
[Itr 2300] Finished
[Itr 2400] Finished
[Itr 2500] Finished
[Itr 2600] Finished
[Itr 2700] Finished
[Itr 2800] Finished
[Itr 2900] Finished
[Itr 3000] Finished
INFO:tensorflow:./Output/cml-yahoo/yahoo-3000 is not in all_model_checkpoint_paths. Manually adding it

In [ ]:
model.save(OUTPUT_PATH,None)

INFO:tensorflow:./Output/cml-yahoo/ is not in all_model_checkpoint_paths. Manually adding it.


In [ ]:
del model

# **DEFINING FUNCTIONS**

In [ ]:
def eq(infilename, infilename_neg, trainfilename, gamma=-1.0, K=1):
    infile = open(infilename, 'rb')
    infile_neg = open(infilename_neg, 'rb')
    P = pickle.load(infile)
    infile.close()
    P_neg = pickle.load(infile_neg)
    infile_neg.close()
    NUM_NEGATIVES = P["num_negatives"]
    #
    for theuser in P["users"]:
        neg_items = list(P_neg["user_items"][theuser][NUM_NEGATIVES:])
        neg_scores = list(P_neg["results"][theuser][NUM_NEGATIVES:])
        P["user_items"][theuser] = list(neg_items) + list(P["user_items"][theuser][NUM_NEGATIVES:])
        P["results"][theuser] = list(neg_scores) + list(P["results"][theuser][NUM_NEGATIVES:])
    #
    Zui = dict()
    Ni = dict()
    # fill in dictionary Ni
    trainset = np.load(trainfilename)
    for i in trainset['item_id']:
        if i in Ni:
            Ni[i] += 1
        else:
            Ni[i] = 1
    del trainset

    # count #users with non-zero item frequencies
    nonzero_user_count = 0
    for theuser in P["users"]:
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        for pos_item in pos_items:
            if pos_item in Ni:
                nonzero_user_count += 1
                break
    # fill in dictionary Zui
    for theuser in P["users"]:
        all_scores = np.array(P["results"][theuser])
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        pos_scores = P["results"][theuser][len(P_neg["results"][theuser][NUM_NEGATIVES:]):]
        for i, pos_item in enumerate(pos_items):
            pos_score = pos_scores[i]
            Zui[(theuser, pos_item)] = float(np.sum(all_scores > pos_score))
    # calculate per-user scores
    sum_user_auc = 0.0
    sum_user_recall = 0.0

    for theuser in P["users"]:
        numerator_auc = 0.0
        numerator_recall = 0.0
        denominator = 0.0
        for theitem in P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]:
            if theitem not in Ni:
                continue
            pui = np.power(Ni[theitem], (gamma + 1) / 2.0)

            numerator_auc += (1 - Zui[(theuser, theitem)] / len(P["user_items"][theuser])) / pui
            # Calcolo il Recall a 1, vedi nota 6 paper
            if Zui[(theuser, theitem)] < K:
                numerator_recall += 1.0 / pui
            denominator += 1 / pui
                

        if denominator > 0:
            sum_user_auc += numerator_auc / denominator
            sum_user_recall += numerator_recall / denominator 

    return {
        "auc"       : sum_user_auc / nonzero_user_count,
        "recall"    : sum_user_recall / nonzero_user_count
    }

In [ ]:
def aoa(infilename, infilename_neg, trainfilename, K=1):
    infile = open(infilename, 'rb')
    infile_neg = open(infilename_neg, 'rb')
    P = pickle.load(infile)
    infile.close()
    P_neg = pickle.load(infile_neg)
    infile_neg.close()
    NUM_NEGATIVES = P["num_negatives"]
    #
    for theuser in P["users"]:
        neg_items = list(P_neg["user_items"][theuser][NUM_NEGATIVES:])
        neg_scores = list(P_neg["results"][theuser][NUM_NEGATIVES:])
        P["user_items"][theuser] = list(neg_items) + list(P["user_items"][theuser][NUM_NEGATIVES:])
        P["results"][theuser] = list(neg_scores) + list(P["results"][theuser][NUM_NEGATIVES:])
    #
    Zui = dict()
    Ni = dict()
    # fill in dictionary Ni
    trainset = np.load(trainfilename)
    for i in trainset['item_id']:
        if i in Ni:
            Ni[i] += 1
        else:
            Ni[i] = 1
    del trainset
    # count #users with non-zero item frequencies
    nonzero_user_count = 0
    for theuser in P["users"]:
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        for pos_item in pos_items:
            if pos_item in Ni:
                nonzero_user_count += 1
                break
    # fill in dictionary Zui
    for theuser in P["users"]:
        all_scores = np.array(P["results"][theuser])
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        pos_scores = P["results"][theuser][len(P_neg["results"][theuser][NUM_NEGATIVES:]):]
        for i, pos_item in enumerate(pos_items):
            pos_score = pos_scores[i]
            Zui[(theuser, pos_item)] = float(np.sum(all_scores > pos_score))
    # calculate per-user scores
    sum_user_auc = 0.0
    sum_user_recall = 0.0
    for theuser in P["users"]:
        numerator_auc = 0.0
        numerator_recall = 0.0
        denominator = 0.0

        for theitem in P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]:
            if theitem not in Ni:
                continue
            numerator_auc += (1 - Zui[(theuser, theitem)] / len(P["user_items"][theuser]))
            # Calcolo il Recall a 30, vedi nota 6 paper
            if Zui[(theuser, theitem)] < K:
                numerator_recall += 1.0
            denominator += 1 

        if denominator > 0:
            sum_user_auc += numerator_auc / denominator
            sum_user_recall += numerator_recall / denominator

    return {
        "auc"       : sum_user_auc / nonzero_user_count,
        "recall"    : sum_user_recall / nonzero_user_count
    }

In [ ]:
def stratified(infilename, infilename_neg, trainfilename, gamma=1.0, K=30, partition=10):

    # Read pickles
    infile = open(infilename, 'rb')
    infile_neg = open(infilename_neg, 'rb')
    P = pickle.load(infile)
    infile.close()
    P_neg = pickle.load(infile_neg)
    infile_neg.close()
    NUM_NEGATIVES = P["num_negatives"]

    # Merge P and P_neg
    for theuser in P["users"]:
        neg_items = list(P_neg["user_items"][theuser][NUM_NEGATIVES:])
        neg_scores = list(P_neg["results"][theuser][NUM_NEGATIVES:])
        P["user_items"][theuser] = list(neg_items) + list(P["user_items"][theuser][NUM_NEGATIVES:])
        P["results"][theuser] = list(neg_scores) + list(P["results"][theuser][NUM_NEGATIVES:])

    Zui = dict()
    Ni = dict()

    # Compute frequencies of items in the training set
    trainset = np.load(trainfilename)
    for i in trainset['item_id']:
        if i in Ni:
            Ni[i] += 1
        else:
            Ni[i] = 1
    del trainset

    # Compute recommendations for each user
    for theuser in P["users"]:
        all_scores = np.array(P["results"][theuser])
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        pos_scores = P["results"][theuser][len(P_neg["results"][theuser][NUM_NEGATIVES:]):]
        for i, pos_item in enumerate(pos_items):
            pos_score = pos_scores[i]
            Zui[(theuser, pos_item)] = float(np.sum(all_scores > pos_score))

    pui = dict()
    w = dict()

    # Compute dictionary of propensity scores
    for theuser in P["users"]:
        for theitem in P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]:
            if theitem not in Ni:
                continue
            if theitem in pui:
                continue
            pui[theitem] = np.power(Ni[theitem], (gamma + 1) / 2.0)

    # Take the list of items (not tuples) in pui sorted by value
    items_sorted_by_value = sorted(pui, key=pui.get, reverse=True)

    # Compute linspace between the pui[0] and pui[-1] 

    # Maybe try to split the logspace instead of the linspace?
    # logspace = np.logspace(pui[items_sorted_by_value[0]], pui[items_sorted_by_value[-1]], partition+1)

    linspace = np.linspace(pui[items_sorted_by_value[0]], pui[items_sorted_by_value[-1]], partition+1)
   
    # Compute dictionary w, that is, for each item, assigns the average of the puis in the partition it belongs to
    i=0
    j = 0
    while i < len(items_sorted_by_value):
                            
        avg = 0
        start = i
        end = i
    
        while i < len(items_sorted_by_value) and pui[items_sorted_by_value[i]] >= linspace[j+1]:
            avg += 1.0 / pui[items_sorted_by_value[i]]
            end = i
            i += 1
        
        # Is the average the only good choice? even with the log space split?
        avg = avg / (end - start + 1)

        for k in range(start, end+1):
            w[items_sorted_by_value[k]] = avg


        j += 1

    nonzero_user_count = 0
    sum_user_auc = 0.0
    sum_user_recall = 0.0

    # Compute score with AUC and compute Recall
    for theuser in P["users"]:
        numerator_auc = 0.0
        numerator_recall = 0.0
        denominator = 0.0
        for theitem in P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]:
            # Skip items with null frequency
            if  theitem not in Ni:
                continue
            # Add things to be summed for each item
            numerator_auc += (1 - Zui[(theuser, theitem)] / len(P["user_items"][theuser])) * w[theitem]
            # Add things for recall
            if Zui[(theuser, theitem)] < K:
                numerator_recall += 1.0 * w[theitem] # spetta
            # Increment denominator that the sum must be divided by 
            denominator += 1 / pui[theitem]


        # If there was at least one item for the user, count the user and sum the results
        if denominator > 0:
            nonzero_user_count += 1
            sum_user_auc += numerator_auc / denominator
            sum_user_recall += numerator_recall / denominator 

    return {
        "auc"       : sum_user_auc / nonzero_user_count, 
        "recall"    : sum_user_recall / nonzero_user_count
    }

In [ ]:
def stratified_logspace(infilename, infilename_neg, trainfilename, gamma=1.0, K=30, partition=10):

    # Read pickles
    infile = open(infilename, 'rb')
    infile_neg = open(infilename_neg, 'rb')
    P = pickle.load(infile)
    infile.close()
    P_neg = pickle.load(infile_neg)
    infile_neg.close()
    NUM_NEGATIVES = P["num_negatives"]

    # Merge P and P_neg
    for theuser in P["users"]:
        neg_items = list(P_neg["user_items"][theuser][NUM_NEGATIVES:])
        neg_scores = list(P_neg["results"][theuser][NUM_NEGATIVES:])
        P["user_items"][theuser] = list(neg_items) + list(P["user_items"][theuser][NUM_NEGATIVES:])
        P["results"][theuser] = list(neg_scores) + list(P["results"][theuser][NUM_NEGATIVES:])

    Zui = dict()
    Ni = dict()

    # Compute frequencies of items in the training set
    trainset = np.load(trainfilename)
    for i in trainset['item_id']:
        if i in Ni:
            Ni[i] += 1
        else:
            Ni[i] = 1
    del trainset

    # Compute recommendations for each user
    for theuser in P["users"]:
        all_scores = np.array(P["results"][theuser])
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        pos_scores = P["results"][theuser][len(P_neg["results"][theuser][NUM_NEGATIVES:]):]
        for i, pos_item in enumerate(pos_items):
            pos_score = pos_scores[i]
            Zui[(theuser, pos_item)] = float(np.sum(all_scores > pos_score))

    pui = dict()
    w = dict()

    # Compute dictionary of propensity scores
    for theuser in P["users"]:
        for theitem in P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]:
            if theitem not in Ni:
                continue
            if theitem in pui:
                continue
            pui[theitem] = np.power(Ni[theitem], (gamma + 1) / 2.0)

    # Take the list of items (not tuples) in pui sorted by value
    items_sorted_by_value = sorted(pui, key=pui.get, reverse=True)

    # Compute linspace between the pui[0] and pui[-1] 

    # Maybe try to split the logspace instead of the linspace?
    logspace = np.logspace(pui[items_sorted_by_value[0]], pui[items_sorted_by_value[-1]], partition+1)
   
    # Compute dictionary w, that is, for each item, assigns the average of the puis in the partition it belongs to
    i=0
    j = 0
    while i < len(items_sorted_by_value):
                            
        avg = 0
        start = i
        end = i
    
        while i < len(items_sorted_by_value) and pui[items_sorted_by_value[i]] >= logspace[j+1]:
            avg += 1.0 / pui[items_sorted_by_value[i]]
            end = i
            i += 1
        
        # Is the average the only good choice? even with the log space split?
        avg = avg / (end - start + 1)

        for k in range(start, end+1):
            w[items_sorted_by_value[k]] = avg


        j += 1

    nonzero_user_count = 0
    sum_user_auc = 0.0
    sum_user_recall = 0.0

    # Compute score with AUC and compute Recall
    for theuser in P["users"]:
        numerator_auc = 0.0
        numerator_recall = 0.0
        denominator = 0.0
        for theitem in P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]:
            # Skip items with null frequency
            if  theitem not in Ni:
                continue
            # Add things to be summed for each item
            numerator_auc += (1 - Zui[(theuser, theitem)] / len(P["user_items"][theuser])) * w[theitem]
            # Add things for recall
            if Zui[(theuser, theitem)] < K:
                numerator_recall += 1.0 * w[theitem] # spetta
            # Increment denominator that the sum must be divided by 
            denominator += 1 / pui[theitem]


        # If there was at least one item for the user, count the user and sum the results
        if denominator > 0:
            nonzero_user_count += 1
            sum_user_auc += numerator_auc / denominator
            sum_user_recall += numerator_recall / denominator 

    return {
        "auc"       : sum_user_auc / nonzero_user_count, 
        "recall"    : sum_user_recall / nonzero_user_count
    }

In [ ]:
def stratified_2(infilename, infilename_neg, trainfilename, gamma=1.0, K=30, partition=10):

    # Read pickles
    infile = open(infilename, 'rb')
    infile_neg = open(infilename_neg, 'rb')
    P = pickle.load(infile)
    infile.close()
    P_neg = pickle.load(infile_neg)
    infile_neg.close()
    NUM_NEGATIVES = P["num_negatives"]

    # Merge P and P_neg
    for theuser in P["users"]:
        neg_items = list(P_neg["user_items"][theuser][NUM_NEGATIVES:])
        neg_scores = list(P_neg["results"][theuser][NUM_NEGATIVES:])
        P["user_items"][theuser] = list(neg_items) + list(P["user_items"][theuser][NUM_NEGATIVES:])
        P["results"][theuser] = list(neg_scores) + list(P["results"][theuser][NUM_NEGATIVES:])

    Zui = dict()
    Ni = dict()

    # Compute frequencies of items in the training set
    trainset = np.load(trainfilename)
    for i in trainset['item_id']:
        if i in Ni:
            Ni[i] += 1
        else:
            Ni[i] = 1
    del trainset

    # Compute recommendations for each user
    for theuser in P["users"]:
        all_scores = np.array(P["results"][theuser])
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        pos_scores = P["results"][theuser][len(P_neg["results"][theuser][NUM_NEGATIVES:]):]
        for i, pos_item in enumerate(pos_items):
            pos_score = pos_scores[i]
            Zui[(theuser, pos_item)] = float(np.sum(all_scores > pos_score))

    pui = dict()
    w = dict()

    # Compute dictionary of propensity scores
    for theuser in P["users"]:
        for theitem in P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]:
            if theitem not in Ni:
                continue
            if theitem in pui:
                continue
            pui[theitem] = np.power(Ni[theitem], (gamma + 1) / 2.0)

    # Take the list of items (not tuples) in pui sorted by value
    items_sorted_by_value = sorted(pui, key=pui.get, reverse=True)

    # Compute linspace between the 0 to len(item_sorted...)
    linspace = np.linspace(0, len(items_sorted_by_value), partition+1)
   
    # Compute dictionary w, that is, for each item, assigns the average of the puis in the partition it belongs to
    i=0
    j = 0
    while i < len(items_sorted_by_value):
                            
        avg = 0
        start = i
        end = i
    
        while i < len(items_sorted_by_value) and i < linspace[j+1]:
            avg += 1.0 / pui[items_sorted_by_value[i]]
            end = i
            i += 1
        
        avg = avg / (end - start + 1)

        for k in range(start, end+1):
            w[items_sorted_by_value[k]] = avg


        j += 1

    nonzero_user_count = 0
    sum_user_auc = 0.0
    sum_user_recall = 0.0

    # Compute score with AUC and compute Recall
    for theuser in P["users"]:
        numerator_auc = 0.0
        numerator_recall = 0.0
        denominator = 0.0
        for theitem in P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]:
            # Skip items with null frequency
            if  theitem not in Ni:
                continue
            # Add things to be summed for each item
            numerator_auc += (1 - Zui[(theuser, theitem)] / len(P["user_items"][theuser])) * w[theitem]
            # Add things for recall
            if Zui[(theuser, theitem)] < K:
                numerator_recall += 1.0 * w[theitem] # spetta
            # Increment denominator that the sum must be divided by 
            denominator += 1 / pui[theitem]


        # If there was at least one item for the user, count the user and sum the results
        if denominator > 0:
            nonzero_user_count += 1
            sum_user_auc += numerator_auc / denominator
            sum_user_recall += numerator_recall / denominator 

    return {
        "auc"       : sum_user_auc / nonzero_user_count, 
        "recall"    : sum_user_recall / nonzero_user_count
    }

In [ ]:
linspace = np.linspace(0, 932, 10+1)
linspace

array([  0. ,  93.2, 186.4, 279.6, 372.8, 466. , 559.2, 652.4, 745.6,
       838.8, 932. ])

# **EVALUATION**

In [ ]:
auc_results = []
recall_results = []

In [ ]:
raw_data = dict()
raw_data['train_data'] = np.load(folder_name + "training_arr.npy")
raw_data['test_data_pos_biased'] = np.load(folder_name + "biased-test_arr_pos.npy")
raw_data['test_data_neg_biased'] = np.load(folder_name + "biased-test_arr_neg.npy")
raw_data['test_data_pos_unbiased'] = np.load(folder_name + "unbiased-test_arr_pos.npy")
raw_data['test_data_neg_unbiased'] = np.load(folder_name + "unbiased-test_arr_neg.npy")
raw_data['max_user'] = 15401
raw_data['max_item'] = 1001
batch_size = 8000
test_batch_size = 1000
display_itr = 1000

train_dataset = ImplicitDataset(raw_data['train_data'], raw_data['max_user'], raw_data['max_item'], name='Train')
test_dataset_pos_biased = ImplicitDataset(raw_data['test_data_pos_biased'], raw_data['max_user'], raw_data['max_item'])
test_dataset_neg_biased = ImplicitDataset(raw_data['test_data_neg_biased'], raw_data['max_user'], raw_data['max_item'])
test_dataset_pos_unbiased = ImplicitDataset(raw_data['test_data_pos_unbiased'], raw_data['max_user'], raw_data['max_item'])
test_dataset_neg_unbiased = ImplicitDataset(raw_data['test_data_neg_unbiased'], raw_data['max_user'], raw_data['max_item'])

In [ ]:
#Code to avoid tf using cached embeddings
import tensorflow as tf
tf.compat.v1.reset_default_graph()

model = MODEL_CLASS(batch_size=batch_size, max_user=train_dataset.max_user(), max_item=train_dataset.max_item(),
    dim_embed=50, l2_reg=0.001, opt='Adam', sess_config=None)
sampler = PairwiseSampler(batch_size=batch_size, dataset=train_dataset, num_process=4)
model_trainer = ImplicitModelTrainer(batch_size=batch_size, test_batch_size=test_batch_size,
                                     train_dataset=train_dataset, model=model, sampler=sampler,
                                     eval_save_prefix=OUTPUT_PATH + DATASET_NAME,
                                     item_serving_size=500)
auc_evaluator = AUC()

model.load(OUTPUT_PATH)

INFO:tensorflow:Restoring parameters from ./Output/cml-yahoo/


In [ ]:
model_trainer._eval_manager = ImplicitEvalManager(evaluators=[auc_evaluator])
model_trainer._num_negatives = 200
model_trainer._exclude_positives([train_dataset, test_dataset_pos_biased, test_dataset_neg_biased])
model_trainer._sample_negatives(seed=10)

[Subsampling negative items]


## Biased Evaluation

In [ ]:
model_trainer._eval_save_prefix = OUTPUT_PREFIX + "-test-pos-biased"
model_trainer._evaluate_partial(test_dataset_pos_biased)

model_trainer._eval_save_prefix = OUTPUT_PREFIX +  "-test-neg-biased"
model_trainer._evaluate_partial(test_dataset_neg_biased)

100%|██████████| 2296/2296 [00:25<00:00, 90.50it/s]


{'AUC': [0.5109661016949152,
  0.5292881355932204,
  0.5090939597315436,
  0.501526845637584,
  0.4819696969696969,
  0.4916610169491526,
  0.5248993288590603,
  0.5493750000000001,
  0.5049832214765101,
  0.49050505050505055,
  0.5272542372881357,
  0.517179054054054,
  0.47917085427135675,
  0.49478333333333335,
  0.5199131944444445,
  0.4848154362416107,
  0.5218729096989966,
  0.4722278911564625,
  0.49442953020134217,
  0.5404208754208754,
  0.5287959866220736,
  0.5320034246575343,
  0.5047826086956522,
  0.45390939597315433,
  0.4978595317725752,
  0.4954401408450704,
  0.5020962199312715,
  0.4976588628762541,
  0.4790816326530612,
  0.5153030303030304,
  0.47370805369127517,
  0.4972558922558923,
  0.4652256944444445,
  0.5101505016722407,
  0.48699664429530204,
  0.5074328859060403,
  0.5444147157190635,
  0.4568027210884354,
  0.4898154362416107,
  0.5067060810810812,
  0.489847972972973,
  0.45332758620689656,
  0.5434129692832764,
  0.4737542087542087,
  0.4798657718120805

## Unbiased Evaluation

In [ ]:
model_trainer._eval_save_prefix = OUTPUT_PREFIX + "-test-pos-unbiased"
model_trainer._evaluate_partial(test_dataset_pos_unbiased)

model_trainer._eval_save_prefix = OUTPUT_PREFIX +  "-test-neg-unbiased"
model_trainer._evaluate_partial(test_dataset_neg_unbiased)

100%|██████████| 2296/2296 [00:01<00:00, 1650.04it/s]


{'AUC': [0.5283333333333333,
  0.5279999999999999,
  0.4749999999999999,
  0.5066666666666666,
  0.3888888888888889,
  0.4685714285714285,
  0.3941666666666666,
  0.5664285714285715,
  0.46375,
  0.37833333333333335,
  0.48562500000000003,
  0.4192857142857144,
  0.5707142857142858,
  0.396875,
  0.468125,
  0.5255555555555556,
  0.5177777777777778,
  0.575625,
  0.5628571428571428,
  0.5327777777777778,
  0.504375,
  0.49666666666666676,
  0.41333333333333333,
  0.5025,
  0.63625,
  0.325,
  0.66,
  0.48812500000000003,
  0.386875,
  0.5288888888888889,
  0.2885714285714286,
  0.5711111111111111,
  0.445,
  0.5122222222222222,
  0.5216666666666667,
  0.6599999999999999,
  0.625,
  0.3608333333333333,
  0.5133333333333333,
  0.3235714285714285,
  0.54875,
  0.41812499999999997,
  0.6425,
  0.59,
  0.4172222222222222,
  0.4475,
  0.5928571428571429,
  0.32916666666666666,
  0.4683333333333333,
  0.3683333333333334,
  0.4316666666666667,
  0.516,
  0.38833333333333336,
  0.43888888888888

## Calculate Metrics

In [ ]:
biased_results = dict()



# biased_results["STRATIFIED_15"] = stratified(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=1.5, K=30, partition=100)
biased_results["AOA"] = aoa(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", K=30)
biased_results["UB_15"] = eq(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=1.5, K=30)
biased_results["UB_2"] =  eq(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=2, K=30)
biased_results["UB_25"] =  eq(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=2.5, K=30)
biased_results["UB_3"] =  eq(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=3, K=30)

In [ ]:
unbiased_results = dict()

# unbiased_results["STRATIFIED_15"] = stratified(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=1.5, K=1, partition=100)
unbiased_results["AOA"] = aoa(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy", K=1)
unbiased_results["UB_15"] = eq(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=1.5, K=1)
unbiased_results["UB_2"] =  eq(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=2, K=1)
unbiased_results["UB_25"] =  eq(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=2.5, K=1)
unbiased_results["UB_3"] =  eq(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=3, K=1)

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
num_items = 932

In [ ]:
nums = np.arange(1, num_items+1)

partitions = np.random.choice(nums, 300, replace=False)
partitions

array([558, 746, 931, 821, 380, 384, 696,  54, 415, 778,  85, 522,  31,
       407, 767, 900, 610,  69, 128, 589, 886, 819, 738, 433, 534,  72,
       607, 217,  93,  48, 468, 764, 168, 637, 898, 140, 238, 615, 430,
       322, 429, 775, 735, 576, 396, 594, 478, 793, 786, 108, 121, 548,
       759, 692, 364, 643, 544,  98, 229, 242, 491, 403, 333, 405, 473,
       573, 711, 795, 492, 909, 184, 620, 236, 827, 716,  59, 349, 675,
       334, 865, 710, 829, 768, 125, 925, 246, 515, 207, 126, 441, 166,
       346, 852, 774, 595, 817, 922, 741, 890, 719,  83, 873, 874, 112,
       352, 665, 434, 916, 555, 297, 831, 897, 171, 210, 582, 417, 398,
       697, 612, 578, 910, 835, 118, 462, 379,  58, 359, 559, 338, 163,
       122, 266,  78, 276, 885, 744, 191, 630, 745, 723, 496, 419, 560,
       557, 878, 836, 820, 507, 851, 627, 661, 460,  94,   2,  39, 727,
       459, 773, 699, 437, 583, 539, 667,  42, 130, 754, 825, 461,  35,
       632, 791, 131, 899, 739, 703, 772, 158, 749, 161, 526, 51

In [ ]:
# Compute biased and unbiased results with stratified for values of partition in (1,2*len(sorted_items))
# and store biased and unbiased results such that abs(biased_results[key]['auc'] - unbiased_results[key]['auc']) + abs(biased_results[key]['recall'] - unbiased_results[key]['recall']) is minimized

#This is the gamma used to compute the best partition
gamma = 2

key = "STRATIFIED_" + str(gamma).replace(".","")

unbiased_results[key] = dict()
biased_results[key] = dict()
best_partition=1

#for p in tqdm(range(1, 2*num_items)):
for p in tqdm(partitions):
    temp_unbiased = stratified(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=gamma, K=1, partition=p)
    temp_biased = stratified(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=gamma, K=30, partition=p)
    if not unbiased_results[key]:
        unbiased_results[key] = temp_unbiased
    if not biased_results[key]:
        biased_results[key] = temp_biased
    elif abs(temp_biased['auc'] - temp_unbiased['auc']) + abs(temp_unbiased['recall'] - temp_biased['recall']) < abs(biased_results[key]['auc'] - unbiased_results[key]['auc']) + abs(biased_results[key]['recall'] - unbiased_results[key]['recall']):
        biased_results[key]['auc'] = temp_biased['auc']
        biased_results[key]['recall'] = temp_biased['recall']
        unbiased_results[key]['auc'] = temp_unbiased['auc']
        unbiased_results[key]['recall'] = temp_unbiased['recall']
        best_partition = p


  0%|          | 0/300 [00:00<?, ?it/s]

In [126]:
best_partition

In [127]:
unbiased_results["STRATIFIED_3"] = stratified(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=3, K=1, partition=best_partition)
biased_results["STRATIFIED_3"] = stratified(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=3, K=30, partition=best_partition)

unbiased_results["STRATIFIED_2"] = stratified(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=2, K=1, partition=best_partition)
biased_results["STRATIFIED_2"] = stratified(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=2, K=30, partition=best_partition)

unbiased_results["STRATIFIED_25"] = stratified(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=2.5, K=1, partition=best_partition)
biased_results["STRATIFIED_25"] = stratified(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=2.5, K=30, partition=best_partition)

unbiased_results["STRATIFIED_15"] = stratified(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=1.5, K=1, partition=best_partition)
biased_results["STRATIFIED_15"] = stratified(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=1.5, K=30, partition=best_partition)
    

In [128]:
unbiased_results["STRATIFIED_v2_3"] = stratified_2(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=3, K=1, partition=best_partition)
biased_results["STRATIFIED_v2_3"] = stratified_2(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=3, K=30, partition=best_partition)

unbiased_results["STRATIFIED_v2_25"] = stratified_2(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=2.5, K=1, partition=best_partition)
biased_results["STRATIFIED_v2_25"] = stratified_2(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=2.5, K=30, partition=best_partition)

unbiased_results["STRATIFIED_v2_2"] = stratified_2(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=2, K=1, partition=best_partition)
biased_results["STRATIFIED_v2_2"] = stratified_2(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=2, K=30, partition=best_partition)

unbiased_results["STRATIFIED_v2_15"] = stratified_2(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=1.5, K=1, partition=best_partition)
biased_results["STRATIFIED_v2_15"] = stratified_2(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=1.5, K=30, partition=best_partition)


In [129]:
key, value = random.choice(list(biased_results.items()))
rows = 2#len(list(value.keys()))
columns = 13#len(list(biased_results.items()))
results_array = np.zeros((rows,columns))

In [130]:
mae_results = dict()

list_biased_res = list(biased_results.keys())

for i in range(len(list_biased_res)):
    key = list_biased_res[i]

    for j in range(len(list(biased_results[key].keys()))):
        key_2 = list(biased_results[key].keys())[j]

        results_array[j][i] = abs(biased_results[key][key_2] - unbiased_results[key][key_2])


In [131]:
mae_df = pd.DataFrame(columns=list(biased_results.keys()), data=results_array)

In [132]:
metric_values = list(biased_results[list(biased_results.keys())[0]].keys())
mae_df.insert(0, "metric", metric_values)


# **RESULTS**

In [133]:
mae_df.head()

,metric,AOA,UB_15,UB_2,UB_25,UB_3,STRATIFIED_15,STRATIFIED_2,STRATIFIED_25,STRATIFIED_3,STRATIFIED_v2_3,STRATIFIED_v2_2,STRATIFIED_v2_25,STRATIFIED_v2_15
0,auc,0.149395,0.122682,0.119498,0.117111,0.115319,0.119082,0.099087,0.037745,0.257602,0.115319,0.119498,0.117111,0.122682
1,recall,0.379472,0.262428,0.251320,0.243180,0.237160,0.261659,0.257094,0.257288,0.221863,0.237160,0.251320,0.243180,0.262428
